# Lab 1
## **Text processing**

## Exercise 1:
Benchmark different language-detection algorithm by computing accuracy of each approach.
- FastText
- LangID
- langDetect

Hint: use language code conversion `iso639-lang`

Report
- Accuracy
- Average time per example

In [ ]:
!wget https://raw.githubusercontent.com/MorenoLaQuatra/DeepNLP/main/practices/P1/langid_dataset.csv

### My code..

In [ ]:
# import
import pandas as pd

# read file
corpus = pd.read_csv('langid_dataset.csv')

In [ ]:
# sneak peak
corpus.head()

In [ ]:
# accuracy function and format printing-function

def acc(clf_fcn, corpus):
    correct = 0
    for text,lan in zip(corpus.Text, corpus.language):
        try:
            pred = clf_fcn(text)
            if type(pred) == tuple: # langid_classify returns lang and prob
                pred = pred[0]
        except:
            continue
        else:
            correct += (pred == Lang(lan).pt1)*1
    return correct

def output(method, corpus, correct, elapsed):
    print(f'{method:s} \nAccuracy: {correct/len(corpus):.3f}. Est time/sample: {elapsed/len(corpus)*1000:.3f} ms')

In [ ]:
import time
#!pip install iso639-lang
from iso639 import Lang

In [ ]:
# FastText (fastlangid)
from fastlangid import LID

method = 'FastText'

# fastText model
fastText_clf = LID()

# accuracy and timing
start = time.time()
correct = acc(fastText_clf.predict, corpus)
elapsed = time.time() - start

# print result
output('FastText', corpus, correct, elapsed)

In [ ]:
# LangID
#!pip install langid
import langid

method = 'LangID'

# accuracy and timing
start = time.time()
correct = acc(langid.classify, corpus)
elapsed = time.time() - start

# print result
output(method, corpus,correct, elapsed)


In [ ]:
# langdetect
#!pip install langdetect
from langdetect import detect

method = 'langdetect'

# accuracy and timing
start = time.time()
correct = acc(detect, corpus)
elapsed = time.time() - start

# print result
output(method, corpus, correct, elapsed)

## Exercise 2
For English-written text, apply word-level tokenization. What is the average number of words per sentence?
Implement word-tokenization using both nltk and spacy. Report the results for both of them.
For spaCy use the en_core_web_sm model.

### My code...

In [ ]:
# find only english texts
corpus_eng = corpus.loc[corpus.language=='English']

In [ ]:
# imports
#!pip install nltk
import nltk
!pip install -U spacy
#python -m spacy download en_core_web_sm (run in terminal)
import spacy

In [ ]:
# counting average number of words

# nltk
tot_words = 0
for sentence in corpus_eng.Text:
    tokens = nltk.word_tokenize(sentence)
    tot_words += len(tokens)

print(f'NLTK \nAverage number of words/sentence: {tot_words/len(corpus_eng):.2f}')

# spacy
spacy_nlp = spacy.load("en_core_web_sm")
tot_words = 0
for sentence in corpus_eng.Text:
    doc = spacy_nlp(sentence)
    sentence_words = 0
    sentence_words += sum([1 for w in doc]) #includes "space" and " - "
    tot_words += sentence_words

print(f'spaCy \nAverage number of words/sentence: {tot_words/len(corpus_eng):.2f}')



In [ ]:
for w in doc:
    print(w)

n = 0
a = sum([1 for w in doc])

In [ ]:
a